## IMAGE CLASSIFICATION

In [21]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.preprocessing import image

## GPU config

In [57]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

ERROR:tensorflow:An interactive session is already active. This can cause out-of-memory errors or some other unexpected errors (due to the unpredictable timing of garbage collection) in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s). Please use `tf.Session()` if you intend to productionize.


 ## Data Preprocessing

In [25]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2,
                                   zoom_range=0.2, horizontal_flip=True)
training_set = train_datagen.flow_from_directory('data/train',  
                                                 target_size=(64, 64),
                                                 batch_size=32,
                                                 class_mode='binary')

test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory('data/test',
                                            target_size=(64, 64),
                                            batch_size=32,
                                            class_mode='binary')

Found 20001 images belonging to 2 classes.
Found 5002 images belonging to 3 classes.


## Building the CNN

In [31]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu', input_shape=(64, 64, 3)))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2))

cnn.add(tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(128, activation='relu'))

# Final output layer for binary classification
cnn.add(tf.keras.layers.Dense(1, activation='sigmoid'))

cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [32]:
cnn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 29, 29, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │         802,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 813,217 (3.10 MB)

 Trainable params: 813,217 (3.10 MB)

 Non-trainable params: 0 (0.00 B)

## Training

In [33]:
cnn.fit(training_set, validation_data=test_set, epochs=15)

Epoch 1/15
626/626 ━━━━━━━━━━━━━━━━━━━━ 75s 116ms/step - accuracy: 0.5851 - loss: 0.6656 - val_accuracy: 0.1583 - val_loss: 0.2375
Epoch 2/15
626/626 ━━━━━━━━━━━━━━━━━━━━ 69s 110ms/step - accuracy: 0.7079 - loss: 0.5636 - val_accuracy: 0.1805 - val_loss: 0.0097
Epoch 3/15
626/626 ━━━━━━━━━━━━━━━━━━━━ 69s 111ms/step - accuracy: 0.7431 - loss: 0.5167 - val_accuracy: 0.1124 - val_loss: 0.2168
Epoch 4/15
626/626 ━━━━━━━━━━━━━━━━━━━━ 68s 109ms/step - accuracy: 0.7682 - loss: 0.4817 - val_accuracy: 0.0840 - val_loss: 0.5251
Epoch 5/15
626/626 ━━━━━━━━━━━━━━━━━━━━ 68s 109ms/step - accuracy: 0.7848 - loss: 0.4525 - val_accuracy: 0.1068 - val_loss: 0.1632
Epoch 6/15
626/626 ━━━━━━━━━━━━━━━━━━━━ 68s 109ms/step - accuracy: 0.7948 - loss: 0.4374 - val_accuracy: 0.1323 - val_loss: -0.2531
Epoch 7/15
626/626 ━━━━━━━━━━━━━━━━━━━━ 67s 108ms/step - accuracy: 0.8044 - loss: 0.4191 - val_accuracy: 0.0984 - val_loss: 0.0553
Epoch 8/15
626/626 ━━━━━━━━━━━━━━━━━━━━ 68s 108ms/step - accuracy: 0.8141 - loss: 

## Saving the model

In [34]:
cnn.save('model_cat_dog.keras')

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the Saved Model
model = load_model('model_cat_dog.keras')

In [40]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 29, 29, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │         802,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,439,653 (9.31 MB)

 Trainable params: 813,217 (3.10 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,626,436 (6.20 MB)

## TESTING

In [65]:
img_path = 'data/test/cats/cat.4021.jpg' 
img = image.load_img(img_path, target_size=(64, 64))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Make Prediction
result = model.predict(img_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


In [66]:
img_path = 'data/test/dogs/dog.4015.jpg' 
img = image.load_img(img_path, target_size=(64, 64))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Make Prediction
result = model.predict(img_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


In [67]:
# Interpret Result
if result[0][0] < 0.5:
    print(f"Predicted: Cat  ({result[0][0]})")
else:
    print(f"Predicted: Dog  ({result[0][0]})")


Predicted: Dog  (0.996867835521698)
